In [0]:
#setting up lakeFS end point access and secret in order to later configure the python client
lakefsEndPoint = 'https://YourEndPoint/' # e.g. 'https://username.azure_region_name.lakefscloud.io'
lakefsAccessKey = 'AKIAlakeFSAccessKey'
lakefsSecretKey = 'lakeFSSecretKey'

In [0]:
# Configuring Python Client

%xmode Minimal
import lakefs_client
from lakefs_client import models
from lakefs_client.client import LakeFSClient

# lakeFS credentials and endpoint
configuration = lakefs_client.Configuration()
configuration.username = lakefsAccessKey
configuration.password = lakefsSecretKey
configuration.host = lakefsEndPoint

client = LakeFSClient(configuration)

In [0]:
# Configuring environment variables

repo = "learn-lakefs-repo01" # the name of the lakeFS repository
storageNamespace = "https://storage-account-name.blob.core.windows.net/storage-container-name/learn-lakefs-repo01" # Unique per repository
sourceBranch = "main"
dataPath = "product-reviews"

In [0]:
# Creating a repository 
client.repositories.create_repository(
    repository_creation=models.RepositoryCreation(
        name=repo,
        storage_namespace=storageNamespace,
        default_branch=sourceBranch))

In [0]:
# Reading data from databricks datasets and inserging into the newly created repository (Creating initial data)

import_data_path = "/databricks-datasets/amazon/test4K/"
df = spark.read.parquet(import_data_path)
df.write.format("parquet").save("lakefs://{}/{}/{}".format(repo,sourceBranch,dataPath))


In [0]:
# Commiting the changes

client.commits.commit(
    repository=repo,
    branch=sourceBranch,
    commit_creation=models.CommitCreation(
        message='Uploading intial data into lakefs',
        metadata={'using': 'python_api'}))

In [0]:
# Review production Data from your production "main" branch

# Note - This example uses static strings instead of parameters for an easier read

df = spark.read.parquet("lakefs://learn-lakefs-repo01/main/product-reviews/")
df.show()

#### More Questions?

###### Join the [lakeFS Slack group](https://lakefs.io/slack)